In [ ]:
import torch
loss = [1,2,3,4]
new_loss = []
alpha =0.5
for _ in loss:
    new_loss.append(sum(_ + torch.tensor(loss)*alpha).item())
new_loss

AssertionError: Torch not compiled with CUDA enabled

In [16]:
import torch
model1 = torch.load("/Applications/codingforlife/LabNLP/LED/sharpseq/checkpoint/model.1", map_location=torch.device('cpu'))
model1.keys()

dict_keys(['state_dict', 'optimizer_state_dict', 'scheduler_state_dict', 'iter'])

In [17]:
model1['iter']

8

In [18]:
import sys
sys.path.append("/Applications/codingforlife/LabNLP/LED/sharpseq")

In [1]:
import os
import json
import glob
from utils.options import Config, PERM
def parse_arguments():
    # parser = argparse.ArgumentParser()
    # define_arguments(parser)
    # args = parser.parse_args()
    args = Config(train_epoch=2,
                  load_model="./checkpoint",
                  json_root="./",
                  feature_root='/Applications/codingforlife/LabNLP/LED/data/features',
                  stream_file='/Applications/codingforlife/LabNLP/LED/data/MAVEN/streams.json',
                  log_dir='./log')
    
    args.log = os.path.join(args.log_dir, "logfile.log")

    if (not args.test_only) and os.path.exists(args.log_dir):
        existing_logs = glob.glob(os.path.join(args.log_dir, "*"))
        for _t in existing_logs:
            if 'exp.log' not in _t and not _t.endswith('.py'):
                os.remove(_t)
    print('Dump name space')
    if not os.path.exists(args.log_dir):
        os.makedirs(args.log_dir)
    with open(f'{args.log_dir}/options.json', 'w') as f:
        print(f'{args.log_dir}/options.json')
        
        json.dump(vars(args), f, ensure_ascii=False)

    return args

In [2]:
from utils.datastream import get_stage_loaders, get_stage_loaders_n

opts = parse_arguments()
perm_id = opts.perm_id
dataset_id = 0
streams = json.load(open(opts.stream_file))
streams = [streams[t] for t in PERM[perm_id]]


loaders, exemplar_loaders, stage_labels, label2id = get_stage_loaders(root=opts.json_root,
    feature_root=opts.feature_root,
    batch_size=opts.batch_size,
    streams=streams,
    num_workers=0,
    dataset=dataset_id)

Dump name space


OSError: [Errno 30] Read-only file system: '/kaggle'

In [ ]:
import torch
from torch.utils.data.dataloader import DataLoader
from torch.utils.data.dataset import TensorDataset
import numpy as np
import json
import os
from tqdm import tqdm

from utils.optimizer import AdamW
from torch.optim import SGD, Adagrad
from utils.options import parse_arguments, PERM
from utils.datastream import get_stage_loaders, get_stage_loaders_n
from utils.worker import Worker
from models.nets import LInEx, BIC, ICARL

summary = None
torch.set_num_threads(20)
def add_summary_value(writer, key, value, iteration):
    return

def by_class(preds, labels, learned_labels=None):
    match = (preds == labels).float()
    nlabels = max(torch.max(labels).item(), torch.max(preds).item())
    bc = {}

    ag = 0; ad = 0; am = 0
    for label in range(1, nlabels+1):
        lg = (labels==label); ld = (preds==label)
        lr = torch.sum(match[lg]) / torch.sum(lg.float())
        lp = torch.sum(match[ld]) / torch.sum(ld.float())
        lf = 2 * lr * lp / (lr + lp)
        if torch.isnan(lf):
            bc[label] = (0, 0, 0)
        else:
            bc[label] = (lp.item(), lr.item(), lf.item())
        if learned_labels is not None and label in learned_labels:
            ag += lg.float().sum()
            ad += ld.float().sum()
            am += match[lg].sum()
    if learned_labels is None:
        ag = (labels!=0); ad = (preds!=0)
        sum_ad = torch.sum(ag.float())
        if sum_ad == 0:
            ap = ar = 0
        else:
            ar = torch.sum(match[ag]) / torch.sum(ag.float())
            ap = torch.sum(match[ad]) / torch.sum(ad.float())
    else:
        if ad == 0:
            ap = ar = 0
        else:
            ar = am / ag; ap = am / ad
    if ap == 0:
        af = ap = ar = 0
    else:
        af = 2 * ar * ap / (ar + ap)
        af = af.item(); ar = ar.item(); ap = ap.item()
    return bc, (ap, ar, af)

def main():
    
    opts = parse_arguments()
    torch.manual_seed(opts.seed)
    np.random.seed(opts.seed)

    dataset_id = 0

    perm_id = opts.perm_id
    if opts.setting == "classic":
        streams = json.load(open(opts.stream_file))
        streams = [streams[t] for t in PERM[perm_id]]
        loaders, exemplar_loaders, stage_labels, label2id = get_stage_loaders(root=opts.json_root,
            feature_root=opts.feature_root,
            batch_size=opts.batch_size,
            streams=streams,
            num_workers=0,
            dataset=dataset_id)

    model = LInEx(
        input_dim=opts.input_dim,
        hidden_dim=opts.hidden_dim,
        max_slots=opts.max_slots,
        init_slots=max(stage_labels[0])+1 if not opts.test_only else max(stage_labels[-1])+1,
        device=torch.device(torch.device(f'cuda:{opts.gpu}' if torch.cuda.is_available() and (not opts.no_gpu) else 'cpu')),
        dropout_type = opts.dropout,
        p = opts.p
        )
    param_groups = [
        {"params": [param for name, param in model.named_parameters() if param.requires_grad and 'correction' not in name],
        "lr":opts.learning_rate,
        "weight_decay": opts.decay,
        "betas": (0.9, 0.999)}
        ]
    optimizer = AdamW(params=param_groups)
    optimizer_correction = None

    worker = Worker(opts)
    worker._log(str(opts))
    worker._log(str(label2id))
    if opts.test_only:
        worker.load(model)
    best_dev = best_test = None
    collect_stats = "accuracy"
    collect_outputs = {"prediction", "label"}
    termination = False
    patience = opts.patience
    no_better = 0
    loader_id = 0
    total_epoch = 0
    none_mul = 4
    learned_labels = set(stage_labels[0])
    dev_metrics = None
    test_metrics = None
    print("Finetune and Generate",opts.finetune, opts.generate)
    while not termination:
        if not opts.test_only:
            if opts.skip_first and loader_id == 0:
                worker.load(model, optimizer, path=opts.load_first, strict=opts.balance!='bic')
                total_epoch += worker.epoch
            elif opts.skip_second and loader_id == 1:
                worker.load(model, optimizer, path=opts.load_second, strict=opts.balance!='bic')
                total_epoch += worker.epoch
            else:
                if opts.finetune:
                    train_loss = lambda batch: model.forward(batch)
                else:
                    train_loss = lambda batch: model.forward(batch, contrastive=opts.contrastive, return_loss_list=opts.mul_task, generate=opts.generate, sample_size=opts.sample_size,
                                                            exemplar=True, exemplar_distill=True, distill=True, tau=0.5,
                                                            feature_distill=opts.features_distill, hyer_distill=opts.hyer_distill, mul_distill=opts.mul_distill,
                                                            lambda_coef=1e-2)
                epoch_loss, epoch_metric = worker.run_one_epoch(
                        model=model,
                        f_loss=train_loss,
                        loader=loaders[loader_id],
                        split="train",
                        optimizer=optimizer,
                        collect_stats=collect_stats,
                        prog=loader_id)
                
                total_epoch += 1
                # if opts.debug:
                #     import pdb
                #     pdb.set_trace()
                for output_log in [print, worker._log]:
                    output_log(
                        f"Epoch {worker.epoch:3d}  Train Loss {epoch_loss} {epoch_metric}")
                # torch.save(model.state_dict(), )
        else:
            learned_labels = set([t for stream in stage_labels for t in stream])
            termination = True


        score_fn = model.score

        dev_loss, dev_metrics = worker.run_one_epoch(
            model=model,
            f_loss=score_fn,
            loader=loaders[-2],
            split="dev",
            collect_stats=collect_stats,
            collect_outputs=collect_outputs)
        dev_outputs = {k: torch.cat(v, dim=0) for k,v in worker.epoch_outputs.items()}
        dev_scores, (dev_p, dev_r, dev_f) = by_class(dev_outputs["prediction"], dev_outputs["label"], learned_labels=learned_labels)
        dev_class_f1 = {k: dev_scores[k][2] for k in dev_scores}
        for k,v in dev_class_f1.items():
            add_summary_value(summary, f"dev_class_{k}", v, total_epoch)
        dev_metrics = dev_f
        for output_log in [print, worker._log]:
            output_log(
                f"Epoch {worker.epoch:3d}:  Dev {dev_metrics}"
            )
        # if worker.epoch == opts.train_epoch:
        #import pdb
        #pdb.set_trace()
        if opts.test_only:
            frequency = {}
            for loader in loaders[:-2]:
                indices = loader.dataset.label2index
                for label in indices.keys():
                    if label != 0:
                        frequency[label] = indices[label][1] - indices[label][0]
            with open("/kaggle/working/sharpseq/data/MAVEN/label2id.json") as fp:
                name2label = json.load(fp)
                label2name = {v:k for k,v in name2label.items()}
            id2label = {v:k for k,v in label2id.items()}
            sf = [(frequency[l], label2name[id2label[l] ], dev_class_f1[l], test_class_f1[l]) for l in frequency]
            sf.sort(key=lambda t:t[0])
            print("macro:", sum([t[3] for t in sf]) / len(sf))

        if not opts.test_only:
            if best_dev is None or dev_metrics > best_dev:
                test_loss, test_metrics = worker.run_one_epoch(
                    model=model,
                    f_loss=score_fn,
                    loader=loaders[-1],
                    split="test",
                    collect_stats=collect_stats,
                    collect_outputs=collect_outputs)
                test_outputs = {k: torch.cat(v, dim=0) for k,v in worker.epoch_outputs.items()}
                # if opts.debug:
                #     import pdb
                #     pdb.set_trace()
                # torch.save(test_outputs, f"/kaggle/working/log/{os.path.basename(opts.load_model)}.output")
                torch.save(test_outputs, f"/kaggle/working/log/test_outputs.output")
                test_scores, (test_p, test_r, test_f) = by_class(test_outputs["prediction"], test_outputs["label"], learned_labels=learned_labels)
                test_class_f1 = {k: test_scores[k][2] for k in test_scores}
                for k,v in test_class_f1.items():
                    add_summary_value(summary, f"test_class_{k}", v, total_epoch)
                test_metrics = test_f
                for output_log in [print, worker._log]:
                    output_log(
                        f"Epoch {worker.epoch:3d}: Test {test_metrics}"
                    )
                best_dev = dev_metrics
                worker.save(model, optimizer, postfix=str(loader_id))
                best_test = test_metrics
                no_better = 0
            else:
                no_better += 1

            print(f"patience: {no_better} / {patience}")
            try:
                with open("/kaggle/working/log/alpha.txt", "a+") as f:
                    f.write(torch.exp(model.input_map[3].log_alpha).__str__())
                    f.write("\n")
            except:
                pass

            if (no_better == patience) or (worker.epoch == worker.train_epoch) or (opts.skip_first and loader_id == 0) or (opts.skip_second and loader_id == 1):
                
 
                loader_id += 1
                #model.id2task[0] = loader_id
                no_better = 0
                worker.load(model, optimizer, path=os.path.join(opts.log_dir, f"{worker.save_model}.{loader_id-1}"))
                if not opts.finetune:
                    print("setting train exemplar for learned classes")
                    #model.set_clusters(exemplar_loaders[loader_id-1])
                    model.set_exemplar(exemplar_loaders[loader_id-1], generate_ratio=opts.generate_ratio, generate=opts.generate, center_ratio=opts.center_ratio, mode=opts.mode, num_clusters=opts.clusters)
                    print("SET EXAMPLAR")
                model.set_class()
                model.lm_head.expand(model.nslots)

                if not opts.finetune:
                    model.set_history()
                    print("SET HISTORY")
                for output_log in [print, worker._log]:
                    output_log(f"BEST DEV {loader_id-1}: {best_dev if best_dev is not None else 0}")
                    output_log(f"BEST TEST {loader_id-1}: {best_test if best_test is not None else 0}")
                if loader_id == len(loaders) - 2:
                    termination = True
                else:
                    learned_labels = learned_labels.union(set(stage_labels[loader_id]))
                    if opts.kt:
                        next_exemplar = model.set_exemplar(exemplar_loaders[loader_id], mode=opts.kt_mode, output_only=True)
                        next_frequency = {}
                        indices = loaders[loader_id].dataset.label2index
                        for label in stage_labels[loader_id]:
                            if label != 0:
                                next_frequency[label] = indices[label][1] - indices[label][0]
                        if opts.kt2:
                            next_inits = model.initialize2(
                                exemplar=next_exemplar,
                                ninstances=next_frequency,
                                gamma=opts.kt_gamma,
                                tau=opts.kt_tau,
                                alpha=opts.kt_alpha,
                                delta=opts.kt_delta)
                        else:
                            next_inits = model.initialize(
                                exemplar=next_exemplar,
                                ninstances=next_frequency,
                                gamma=opts.kt_gamma,
                                tau=opts.kt_tau,
                                alpha=opts.kt_alpha)
                        torch.save(model.outputs["new2old"], os.path.join(opts.log_dir, f"{loader_id}_to_{loader_id-1}"))
                        model.extend(next_inits)
                        assert model.nslots == max(learned_labels) + 1
                    else:
                        model.nslots = max(learned_labels) + 1
                worker.epoch = 0
                best_dev = None; best_test = None

if __name__ == "__main__":
    main()
